# Week4 Homework1 Graph Neural Network


# Check GPU Type

In [ ]:
!nvidia-smi

Sun Sep 22 06:31:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Download Data


In [ ]:
!pip install gdown --upgrade
!gdown --id '1jp5XRat8h3w4fZ7nEmDXBCf7-dfUFl1S' --output data.zip

  Attempting uninstall: gdown
    Found existing installation: gdown 5.1.0
    Uninstalling gdown-5.1.0:
      Successfully uninstalled gdown-5.1.0
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1jp5XRat8h3w4fZ7nEmDXBCf7-dfUFl1S
From (redirected): https://drive.google.com/uc?id=1jp5XRat8h3w4fZ7nEmDXBCf7-dfUFl1S&confirm=t&uuid=63662de0-814c-456f-a474-1723f937cadb
To: /content/data.zip
100% 61.9M/61.9M [00:01<00:00, 46.8MB/s]


In [ ]:
! unzip data.zip

Archive:  data.zip
   creating: dataset/
  inflating: dataset/features_test.pkl  
  inflating: dataset/features_train.pkl  
  inflating: dataset/graph_test.pkl  
  inflating: dataset/graph_train.pkl  


# Import Packages

In [ ]:
! pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00


In [ ]:
import easydict
import numpy as np
import pandas as pd
import pickle as pkl
import json
import math
import copy, os

import scipy.sparse as sp
from scipy.stats import rankdata
from sklearn.metrics import mean_absolute_error

import torch
import torch_geometric as tg
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.nn.modules.loss import MSELoss
from torch_geometric.nn.conv import GATConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
torch.autograd.set_detect_anomaly(True)

# Usable Functions

We use these functions to calculate the loss so we can better tune our model.

In [ ]:
def check_directories(directories):
    for directory in directories:
        if not os.path.exists("./" + directory):
            os.makedirs("./" + directory)

def to_device(batch, device):
    data = copy.deepcopy(batch)
    idx, graphs, labels = data.values()
    data["idx"] = idx[0].to(device)
    data["hist_graphs"] = [x.to(device) for x in graphs]
    data["labels"] = labels[0].to(device)
    return data

def mape_clip(label, pred, threshold = 0.1):
    v = np.clip(np.abs(label), threshold, None)
    diff = np.abs((label - pred) / v)
    return 100.0 * np.mean(diff)

def rankic(y_true, y_pred):
    rank_true = len(y_true) - rankdata(y_true)+1
    rank_pred = len(y_pred) - rankdata(y_pred)+1
    res = np.corrcoef(rank_true, rank_pred)[0,1]
    return res

def calculate_perf(pred, label):
    mae = mean_absolute_error(label, pred)
    mape = mape_clip(label, pred)
    ic = np.corrcoef(label, pred)[0,1]
    rank_ic = rankic(label, pred)
    return [mae, mape, ic, rank_ic]

# Load Datasets

The datasets are saved in pickle format. After loading it, we still need to preprocess the data.

The training dataset contains 94 features, whereas the testing dataset has only 93 features. This discrepancy is due to the first column of the training dataset, which is the label, being removed from the testing dataset.

In [ ]:
class AssetBatch(Dataset):
    def  __init__(self, args, adjs, features, labels,
                  start, end):
        super(AssetBatch, self).__init__()
        self.args = args
        self.start = start
        self.end = end
        self.adjs = adjs
        self.features = features
        self.labels = labels
        self.hist_time_steps = args.hist_time_steps

    def __len__(self):
        return self.end-self.start + 1

    def __getitem__(self, index):
        idx = index+self.start
        hist_graphs = []
        for i in range(idx - self.args.hist_time_steps, idx):
            x = torch.Tensor(self.features[i])
            edge_index, edge_weight = tg.utils.from_scipy_sparse_matrix(self.adjs[i])

            graph = Data(x=x, edge_index=edge_index, edge_weight=edge_weight)
            hist_graphs.append(graph)
        sample = {'idx': torch.Tensor([idx]),
                'hist_graphs': hist_graphs,
                'labels': torch.Tensor(self.labels[idx])}
        return sample

    def subset(self, start_loc, end_loc):
        adjusted_start = self.start + start_loc
        adjusted_end = self.start + end_loc
        return AssetBatch(self.args, self.adjs, self.features, self.labels, adjusted_start, adjusted_end)




class AssetDataset():
    def __init__(self, args, features, adjs, mode="train"):
        super(AssetDataset, self).__init__()
        self.args = args
        self.max_steps = len(adjs)
        self.hist_time_steps = args.hist_time_steps
        if mode == "train":
            self.labels = [self._preprocess_labels(feat) for feat in features]
            self.features = [self._preprocess_features(feat) for feat in features]
            self.adjs = [self._preprocess_adj(a) for a in adjs]
            self._split_data()
        elif mode == "test":
            self.labels = len(features) * [0]
            self.features = [self._preprocess_features(feat) for feat in features]
            self.adjs = [self._preprocess_adj(a) for a in adjs]
            self._test_data()

    def _preprocess_labels(self, features):
        """split return (label) from features"""
        features = np.array(features.todense())
        labels = features[:,0]
        return labels

    def _preprocess_features(self, features):
        features = np.array(features.todense())
        if self.args.feat_norm:
            rowsum = np.array(features.sum(1))
            r_inv = np.zeros_like(rowsum).flatten()
            non_zero_indices = rowsum != 0
            r_inv[non_zero_indices] = np.power(rowsum[non_zero_indices], -1).flatten()
            r_mat_inv = sp.diags(r_inv)
            features = r_mat_inv.dot(features)
            return features
        return features

    def _preprocess_adj(self, adj):
        if self.args.adj_norm:
            """normalization of adjacency matrix (scipy sparse format). Output is in tuple format"""
            rowsum = np.array(adj.sum(1))
            r_inv = sp.diags(np.power(rowsum, -1).flatten(), dtype=np.float32)
            adj_normalized = r_inv.dot(adj)
            return adj_normalized
        return adj

    def _split_data(self):
        train_start = self.args.hist_time_steps
        valid_start = int(np.floor(self.max_steps * self.args.train_proportion))


        train = AssetBatch(self.args, self.adjs, self.features, self.labels,
                           train_start, valid_start-1)
        valid = AssetBatch(self.args, self.adjs, self.features, self.labels,
                           valid_start + self.args.hist_time_steps, self.max_steps-1)

        self.train = DataLoader(train, shuffle=True, batch_size = self.args.batch_size)
        self.valid = DataLoader(valid, shuffle=False)


        print('Dataset splits: ')
        print('{:<3} train samples from {:<3} to {:<3}'.format(len(self.train), train_start, valid_start-1))
        print('{:<3} valid samples from {:<3} to {:<3}'.format(len(self.valid), valid_start + self.args.hist_time_steps, self.max_steps-1))


    def _test_data(self):
        test_start = self.args.hist_time_steps
        test = AssetBatch(self.args, self.adjs, self.features, self.labels,
                           test_start, self.max_steps-1)
        self.test = DataLoader(test, shuffle=False)

        print('Dataset info: ')
        print('{:<3} test samples from {:<3} to {:<3}'.format(len(self.test), test_start, self.max_steps-1))

# Graph Attention Networks

Feel free to edit your version of the model. You can check more details about GAT [here](https://arxiv.org/pdf/1710.10903).

In [ ]:
class GATconv(torch.nn.Module):
    def __init__(self, args, node_features):
        super(GATconv, self).__init__()
        self.args = args
        self.num_time_steps = args.hist_time_steps
        self.num_features = node_features
        self.static = GATConv(node_features, 16, 4)
        self.linear = torch.nn.Linear(64, 1)
        self.mseloss = MSELoss()

    def forward(self, graphs):
        x = []
        for t in range(0, self.num_time_steps):
            snapshot = graphs[t]
            x.append(snapshot.x[:,0])
        x = torch.stack(x, dim=1)
        edge_index = snapshot.edge_index
        edge_weight = snapshot.edge_weight
        h = self.static(x, edge_index)
        y = F.relu(h)
        y = self.linear(y).flatten()
        return y

    def get_loss(self, data, chosen=None): # data: (N)
        idx, graphs, labels = data.values()
        # run gnn
        pred = self.forward(graphs)

        next_pred = pred
        next_labels = labels

        graphloss = self.mseloss(next_pred,next_labels)
        return graphloss, next_pred.detach().cpu().numpy(), next_labels.detach().cpu().numpy()

    def predict(self, data):
        idx, graphs, labels = data.values()

        predictions = self.forward(graphs)
        return predictions.detach().cpu().numpy()

# Args you can use

In [ ]:
args = easydict.EasyDict({
    "hist_time_steps": 12,
    "dataset": "dataset",
    "GPU_ID": 0,
    "model": "GAT",
    "epochs": 3,
    "val_freq": 1,
    "test_freq": 1,
    "batch_size": 1,
    "feat_norm": True,
    "adj_norm": True,
    "early_stop": 30,
    "train_proportion": 0.7,
    "valid_proportion": 0.15,
    "residual": True,
    "learning_rate": 0.0001,
    "spatial_drop": 0.1,
    "temporal_drop": 0.5,
    "weight_decay": 0.0005,
    "node_dim": 8,
    "n_heads": 16,
    "attention_layers": 1,
    "centrality": True,
    "spatial": True,
    "edge": True
})

# Make Directories

This is where you store your results.

In [ ]:
RESULTS_DIR = "result/"
check_directories([RESULTS_DIR])
with open(RESULTS_DIR+'/args.txt', 'w') as f:
    json.dump(args.__dict__, f, indent=2)
print('Results directory:', RESULTS_DIR)

Results directory: result/


# Training

In [ ]:
print("Start Traning...")

with open('dataset/graph_train.pkl', 'rb') as file:
    adjs = pkl.load(file)
with open('dataset/features_train.pkl', 'rb') as file:
    feats = pkl.load(file)

feat_dim = feats[0].shape[1]
num_nodes = adjs[0].shape[0]

print('Total time steps:', len(adjs))
print('Total number of assets:', num_nodes)
print('Total number of features:', feat_dim)

# total time steps used for train, eval and test
assert args.hist_time_steps < len(adjs), "Time steps is illegal"

# build dataloader and model
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
dataloader = AssetDataset(args, feats, adjs, "train")


model = GATconv(args, args.hist_time_steps).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

# training
best_epoch_val, best_epoch = 1000000, 0
patient = 0
for epoch in range(args.epochs):
    model.train()
    epoch_loss = []
    for idx, train_data in enumerate(dataloader.train):
        train = to_device(train_data, device)
        opt.zero_grad()
        loss, _, _ = model.get_loss(train)
        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())

    # get evaluation results
    model.eval()
    eval_loss, eval_rmse, eval_mae, eval_mape = [], [], [], []
    for idx, valid_data in enumerate(dataloader.valid):
        valid = to_device(valid_data, device)
        eval_loss_idx, eval_pred, eval_labels = model.get_loss(valid)
        eval_mae_idx, eval_mape_idx, _, _ = calculate_perf(eval_pred, eval_labels)
        eval_loss.append(eval_loss_idx.detach().cpu().numpy())
        eval_rmse.append(math.sqrt(eval_loss_idx.detach().cpu().numpy()))
        eval_mae.append(eval_mae_idx)
        eval_mape.append(eval_mape_idx)

    if np.mean(eval_loss) < best_epoch_val:
        best_epoch_val = np.mean(eval_loss)
        best_epoch = epoch
        torch.save(model.state_dict(), RESULTS_DIR+"/model.pt")
        patient = 0
    else:
        patient += 1
        if patient > args.early_stop:
            break

    print("Epoch {:<3}, lr = {:<5},  Train Loss = {:.4f}, Valid Loss = {:.4f}, Val MAE = {:.4f}, Val MAPE = {:.4f}, Val RMSE = {:.4f}. ".format(\
            epoch, opt.param_groups[0]["lr"], np.mean(epoch_loss), np.mean(eval_loss),\
            np.mean(eval_mae), np.mean(eval_mape), np.mean(eval_rmse)))

Start Traning...
Total time steps: 175
Total number of assets: 372
Total number of features: 94
Dataset splits: 
110 train samples from 12  to 121
41  valid samples from 134 to 174
Epoch 0  , lr = 0.0001,  Train Loss = 0.0080, Valid Loss = 0.0057, Val MAE = 0.0594, Val MAPE = 51.7502, Val RMSE = 0.0743. 
Epoch 1  , lr = 0.0001,  Train Loss = 0.0079, Valid Loss = 0.0057, Val MAE = 0.0593, Val MAPE = 51.6205, Val RMSE = 0.0741. 
Epoch 2  , lr = 0.0001,  Train Loss = 0.0079, Valid Loss = 0.0057, Val MAE = 0.0590, Val MAPE = 51.4464, Val RMSE = 0.0739. 


# Testing

In [ ]:
print("Start Testing...")
# Prediction by Best Model
model.load_state_dict(torch.load(RESULTS_DIR+"/model.pt"))
model.eval()

with open('dataset/graph_test.pkl', 'rb') as file:
    adjs_test = pkl.load(file)
with open('dataset/features_test.pkl', 'rb') as file:
    feats_test = pkl.load(file)
feat_dim = feats_test[0].shape[1]
num_nodes = adjs_test[0].shape[0]

dataloader = AssetDataset(args, feats_test, adjs_test, "test")


# Test Best Model
model.load_state_dict(torch.load(RESULTS_DIR+"/model.pt"))
model.eval()
test_preds = []
for idx, test_data in enumerate(dataloader.test):
    test = to_device(test_data, device)
    test_pred_idx = model.predict(test)
    test_preds.append(test_pred_idx)

# write to csv
pred = pd.DataFrame(test_preds)
stacked_pred = pred.stack().reset_index(drop=True)
stacked_pred = stacked_pred.reset_index()
stacked_pred.columns = ['Id', 'Label']
stacked_pred.to_csv(RESULTS_DIR+'/test_pred.csv', index=False)

Start Testing...
Dataset info: 
21  test samples from 12  to 32 


<ipython-input-12-ced60d7e01ac>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(RESULTS_DIR+"/model.pt"))
<ipython-input-12-ced60d7e01ac>:1